In [ ]:
from sklearn.datasets import load_files
from nltk import word_tokenize
corpus =  load_files('D:/split/train26/', encoding = 'utf_8' )

sentance=[]
for i in corpus.data:
    sentences = [word_tokenize(s) for s in i.lower().split('\n')]
    sentance += sentences
print(sentance[0])

In [ ]:
from gensim.models import word2vec
model = word2vec.Word2Vec(sentance, size=300, window=20,
                          min_count=2, workers=1, iter=100)
model.corpus_count
vocab = model.wv.vocab


In [ ]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from sklearn.pipeline import Pipeline
TRAIN_SET_PATH = "D:/france/madar_shared_task/MADAR-Shared-Task-Subtask-1/MADAR-Corpus-26-train.txt"
X, yt = [], []
with open(TRAIN_SET_PATH, "r", encoding='utf_8') as infile:
    for line in infile:
        text, label  = line.split("\t")
                # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X.append(text)
        yt.append(label)
wv_train_feat = word2vec_features(X, model)
pip = Pipeline([   ("tfidf_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB(alpha=0.5))
])
#logreg = linear_model.LogisticRegression(n_jobs=1, C=1e5)
#pip.fit(wv_train_feat, yt)

In [19]:
TRAIN_SET_PATH = "D:/france/madar_shared_task/MADAR-Shared-Task-Subtask-1/MADAR-Corpus-26-dev.tsv"
x,y = [],[]
with open(TRAIN_SET_PATH, "r", encoding='utf_8') as infile:
    for line in infile:
        text, label  = line.split("\t")
        x.append(text)
        y.append(label)
wv_test_feat = word2vec_features(x, model)
pip.fit(wv_test_feat, y)

#y_pred = logreg.predict(wv_train_feat)
#from sklearn.metrics import accuracy_score, f1_score, precision_score
#print('Testing accuracy %s' % precision_score(yt, y_pred, average=))


Pipeline(memory=None,
     steps=[('tfidf_vectorizer', CountVectorizer(analyzer=<function <lambda> at 0x000000005BADB598>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), pr..., vocabulary=None)), ('multinomial nb', MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))])

In [20]:
#utilisation logistic regression : sur train se donne 12.52% , et sur dev se donne 12.62% 
pip.score(wv_train_feat, yt)

0.03875